## 講習3 --- pyfitsの基本  

FITSデータの読み書きが可能なモジュールです。  
pyfitsの使い方は、  
http://stsdas.stsci.edu/download/docs/The_PyFITS_Handbook.pdf  
が非常に参考になります。  例が多く記載されているのでわかりやすいです。  

PyRAFでもFITSデータの読み書きはできるのですが、ndarrayとしてデータを読み出しておくとより自由な処理を行うことができます。例えば、PyRAFには存在しない統計アルゴリズムを適用する、あるいは、matplotlibを用いた自由度の高い可視化などです。



In [1]:
import pyfits

まずpyfitsをimportしておきます。pyfitsはあまりas ナントカで省略されることがないようです。  

### 基本のgetdataとgetheader 
まずは簡単なデータ読み出し方法。** pyfits.getdata( )**と **pyfits.getheader( )**です。  主にインタラクティブなケースで使われます。  

まずはピクセル値をgetdataでndarrayとして読み出します。

In [2]:
data = pyfits.getdata('./010604.ffiles/jf0001.fits')

In [3]:
data

array([[-100000.        , -100000.        ,     239.08718872, ...,
            271.50674438,     262.92190552, -100000.        ],
       [-100000.        ,     241.92218018,     253.86941528, ...,
            261.04660034,     253.49819946, -100000.        ],
       [    233.6789856 ,     240.16479492,     274.65298462, ...,
            252.41287231, -100000.        , -100000.        ],
       ..., 
       [-100000.        , -100000.        , -100000.        , ...,
        -100000.        , -100000.        , -100000.        ],
       [-100000.        , -100000.        , -100000.        , ...,
        -100000.        , -100000.        , -100000.        ],
       [-100000.        , -100000.        , -100000.        , ...,
        -100000.        , -100000.        , -100000.        ]], dtype=float32)

ndarrayとして読み出すことができます。なので、numpyの関数が使えます。

In [4]:
import numpy as np

In [5]:
np.median(data), np.std(data), np.min(data), np.max(data)

(252.41287, 16631.922, -199929.28, 20279.666)

pyIRSFのスカイ引き後のイメージなので、badpixelにはマイナスの大きな値が入っています。 なので標準偏差(np.std)は大きな値になっています。ピクセル値が0以上のところだけを取り出して統計をとってみましょう。

In [6]:
xx = np.where(data > 0)
np.median(data[xx]), np.std(data[xx]), np.min(data[xx]), np.max(data[xx])

(252.41287, 64.200676, 22.012028, 20279.666)

次にgetheaderを使ってFITSヘッダを読み出します。  

In [7]:
header = pyfits.getheader('./010604.ffiles/jf0001.fits')

In [8]:
header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -32 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 1024                                                  
NAXIS2  =                 1024                                                  
OBJECT  = 'p9144std'           /  Object Name                                   
FILTER  = 'J       '           /  Filter Name                                   
EXPOS   =                   5. / Exposure time(s)                               
DATE_UTC= '2001/06/04'         / yyyy:mm:dd                                     
TIME_UTC= '16:44:48.8'         / hh:mm:ss                                       
DATE_LT = '2001/06/04'         / yyyy:mm:dd                                     
TIME_LT = '18:44:48.8'         / hh:mm:ss                                       
JD      = '52064.0 '        

In [9]:
header['RA']

'10:47:24.1'

このようにヘッダのキーワードを指定して、その値を取り出すことができます。

#### データの加工と書き出し   
あまり意味のある加工ではありませんが、ピクセル値を全体のmedianで規格化し、ヘッダの'OBJECT'を書き換えたものを新しいファイル'myout.fits'に書き出します。書き出しには**pyfits.append( )**を使います。


In [10]:
data = data / np.median(data[xx])

In [11]:
header['OBJECT'] = 'p9144stdnorm'

In [12]:
pyfits.append('myout.fits', data, header)

ここの例では'./010604.ffiles/jf0001.fits'を読み出して、データの加工をしたものを新しいファイル'myout.fits'に書き出しましたが、元の'./010604.ffiles/jf0001.fits'に上書き更新する場合には**pyfits.update( )**を使います。

pyfits.getdata( ) や pyfits.getheader( )、pyfits.append( )などは簡単で便利(これらはpyfitsの中で'convinience functions'と呼ばれている)なのですが、効率の悪いことをしています。その都度にファイルのオープンとクローズをしています。

### pyfits.open() を使う
一般に、プログラムコードの中でFITSファイルの読み書きをする場合には次のように、** pyfits.open() **でファイルを開き、**.headerメソッド**と**.dataメソッド**を使ってヘッダーとデータを読み取ります。

In [13]:
hdulist = pyfits.open('./010604.ffiles/kf0001.fits')

このFITSファイルがどのような構造になっているかを、**.info()メソッド**を使って調べることができます。

In [14]:
hdulist.info()

Filename: ./010604.ffiles/kf0001.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU      27   (1024, 1024)   float32   


一つのヘッダと画像データのセットだけが含まれています。 
この場合、hdulist[0]のみにデータが含まれており、次のようにヘッダと画像データを読み取ります。

In [15]:
hdr = hdulist[0].header
imdata = hdulist[0].data

拡張FITSファイルの場合、hdulistの要素が複数あり、hdulist[1]などと指定します。   

In [16]:
hdr['OBJECT']

'p9144std'

In [17]:
imdata[500,500]

1029.2842

ヘッダのキーワードのOBJECTを書き換え、全てのピクセル値を2倍にしたものを新しいFITSファイルに保存してみます。

In [18]:
hdr['OBJECT'] = 'changed'
hdulist[0].header = hdr

In [19]:
hdulist[0].data = 2 * imdata

新しいファイルに書き出す場合には、** .writetoメソッド**を使います。 

In [20]:
hdulist.writeto('myout2.fits')

開いたファイルは最後には閉じておきましょう。

In [21]:
hdulist.close()

読み込んだファイルに上書き更新する場合には、 
下のように、openするときにmode='update'を指定しておきます。変更がclose()のときにファイルの中身に書き込まれます。

In [22]:
f = pyfits.open('myout2.fits', mode='update')
data = f[0].data
f[0].data = data * 2.
f.close()

### 人工的な画像  
ピクセル値が全部ゼロの10 x 10のFITSファイルを作り、いくつかのピクセルにだけ正の値を与えてやります。

In [24]:
import numpy as np
import pyfits 

imdata = np.zeros((10, 10), dtype='float32')  # 全て0の10x10のndarrayを作成
imdata[0, 1] = 100. #  (x,y)=(1,0)に100を代入。xとyが反転していることに注意
imdata[4, 6] = 50. 

hdu = pyfits.PrimaryHDU(imdata)
img = pyfits.HDUList([hdu])   #  最低限必要なFITSヘッダも作成される
img.writeto('my10x10.fits')
img.close()

出来上がったFITSファイルをDS9で見てみましょう。

** 練習問題 **  
1. 二つのFITSデータの差分をとり、新しいファイルに書き出してみましょう。